In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from seaborn import color_palette
import numpy as np
from util import load_results_from_dir

# sns.set_palette('rocket_r')
# sns.set_palette('hls')
# colors = ["#5B656F", "#A1B5D8", "#EDCB96", "#DB5461", "#171738", "#6C783B"]
# colors = ["#fd7f6f", "#7eb0d5", "#b2e061", "#bd7ebe", "#ffb55a", "#ffee65", "#beb9db", "#fdcce5", "#8bd3c7"]
# colors = ["#ffb3b8", "#fa4d56", "#9f1853", "#1192e8", "#009d9a", "#005d5d"]
colors = ["#f77e84", "#9f1853", "#1192e8", "#009d9a", "#012749", "#ACACAC"]
sns.set_palette(sns.color_palette(colors))

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        # "font.serif": ["Computer Modern Roman", "Times New Roman"] + plt.rcParams["font.serif"],
        "text.latex.preamble": r"\usepackage[T1]{fontenc} \usepackage[utf8]{inputenc} \usepackage{lmodern}",
    }
)
figsize=np.array((3,1.6)) * 0.9
dpi = 120
parameters = ['scheduler', 'N_w', 'N_d', 'N_s', 'W_hat', 'pi', 'sigma', 'anchor_count', 'epsilon', 'int_feas_tol', 'v', 'r_charge', 'r_deplete', 'B_min', 'B_max', 'B_start', 'trial']

In [ ]:
rootdir = "../out/villalvernia.elrond/grid_search"
df = load_results_from_dir(rootdir)
df = df[lambda x: (x.scheduler == 'naivescheduler') | (x.pi.isin([np.inf, 61, 46, 31, 16, 8]))]
df['t_solve_mean_after_first'] = df.apply(lambda x: np.mean(x.solve_times[1:]), axis=1)
df['t_solve_std_after_first'] = df.apply(lambda x: np.std(x.solve_times[1:]), axis=1)
df = df[lambda x: x.trial == 1]
df = df.sort_values('experiment_timestamp', ascending=False).drop_duplicates(parameters)

In [ ]:
print(f"Size of dataset: {df.shape[0]:,}\n")
df.sort_values(by='experiment_timestamp')[parameters + ['execution_time', 't_solve_mean', 't_solve_total', 'n_solves']]

In [ ]:
naive_execution_time = df[lambda x: x.scheduler == 'naivescheduler'].iloc[0].execution_time

# Plots

In [ ]:
label_translation = {
    'inf': '$\infty$',
    '8.0': '8',
    '16.0': '16',
    '31.0': '31', 
    '46.0': '46',
    '61.0': '61',
}

## Execution time

In [ ]:
cp = color_palette()
ymax = df.execution_time.max() * 1.025
ymin = 3500 # ymin = df.execution_time.min() * 0.7
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='execution_time', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)
    
    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='lower right', edgecolor='white', framealpha=1, borderpad=0, ncol=2)
    ax.axhline(y=naive_execution_time, color='red', linestyle=':')
    if i == 0:
        plt.ylabel("Execution time (s)")
    else:
        plt.ylabel("")
        leg.remove()
    plt.xlabel("$\hat{N}_w$")
    plt.grid(axis='y')
    plt.ylim([ymin, ymax])
    # plt.title(f"$\sigma = {sigma}$")
        
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.execution_time.bar.pdf", bbox_inches='tight', pad_inches=0.02)
    plt.show()

In [ ]:
min_execution_time = df[lambda x: x.scheduler == 'milpscheduler'].execution_time.min()
max_execution_time = df[lambda x: x.scheduler == 'milpscheduler'].execution_time.max()
min_execution_time_rel = (min_execution_time / naive_execution_time * 100) - 100
max_execution_time_rel = (max_execution_time / naive_execution_time * 100) - 100

print(f"Minimum execution time: {min_execution_time:.1f}s")
print(f"Maximum execution time: {max_execution_time:.1f}s")

print(f"Minimum relative: {min_execution_time_rel:.1f}%")
print(f"Maximum relative: {max_execution_time_rel:.1f}%")

In [ ]:
print("Best performing combination of parameters:")
display(df[lambda x: x.scheduler == 'milpscheduler'].sort_values('execution_time').iloc[0][['W_hat', 'sigma', 'pi']].reset_index())

## Scheduling time

### Total

In [ ]:
cp = color_palette()
ymax = df.t_solve_total.max() * 2
ymin = df.t_solve_total[lambda x: x>0].min() / 1.1
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='t_solve_total', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='lower right', edgecolor='white', framealpha=1, borderpad=0, ncol=2)
    if i == 0:
        plt.ylabel("Total scheduling\ntime (s)")
    else:
        plt.ylabel("")
        leg.remove()
    plt.xlabel("$\hat{N}_w$")
    plt.grid(axis='y')
    plt.yscale('log')
    ax.set_ylim([ymin, ymax])
    # plt.title(f"$\sigma = {sigma}$")
    
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.scheduling_time_total.bar.pdf", bbox_inches='tight', pad_inches=0.02)
    plt.show()

### Average

In [ ]:
cp = color_palette()
ymax = df.t_solve_mean.max() * 2
ymin = df.t_solve_mean[lambda x: x>0].min() / 1.1
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    print(sigma)
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='t_solve_mean', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='lower right', edgecolor='white', framealpha=1, borderpad=0, ncol=2)
    if i == 0:
        plt.ylabel("Mean scheduling\ntime (s)")
    else:
        plt.ylabel("")
        leg.remove()
    
    plt.xlabel("$\hat{N}_w$")
    plt.grid(axis='y')
    ax.set_yscale('log')
    ax.set_ylim([ymin, ymax])
    # plt.title(f"$\sigma = {sigma}$")
    
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.scheduling_time_mean.bar.pdf", bbox_inches='tight', pad_inches=0.02)
    plt.show()

# Table format

In [ ]:
def formatter(x):    
    suffix = "}"
    if x > 0:
        # red
        prefix = r"\rednr{"
    else:
        # green
        prefix = r"\greennr{"
    return prefix + f"{x:.1f}" + suffix

In [ ]:
W_hats = df[lambda x: x.scheduler == 'milpscheduler'].W_hat.unique()
pis = df[lambda x: x.scheduler == 'milpscheduler'].pi.unique()

## Execution time

In [ ]:
min_execution_time = df[lambda x: x.scheduler == 'milpscheduler'].execution_time.min()

In [ ]:
df_plots = []

for sigma in [1, 2, 3, 4]:
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != 'naivescheduler')].pivot('pi', 'W_hat', 'execution_time') / min_execution_time * 100 - 100
    df_plot = df_plot.reindex(pis, axis=0)
    display(df_plot)
    df_plots.append(df_plot)    
df_execution_time = pd.concat(df_plots, axis=1)
print(df_execution_time.to_latex(na_rep="", formatters=[formatter] * len(df_execution_time.columns), escape=False, index_names=False))

## Scheduling time

### Total

In [ ]:
df_plots = []

for sigma in [1, 2, 3, 4]:
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != 'naivescheduler')].pivot('pi', 'W_hat', 't_solve_total')
    df_plot = df_plot.reindex(pis, axis=0)
    display(df_plot)
    df_plots.append(df_plot)    
df_scheduling_time_total = pd.concat(df_plots, axis=1)
print(df_scheduling_time_total.to_latex(na_rep="", float_format="%.0f", escape=False, index_names=False))

### Average rescheduling time

In [ ]:
df_plots = []

for sigma in [1, 2, 3, 4]:
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != 'naivescheduler')].pivot('pi', 'W_hat', 't_solve_mean_after_first')
    df_plot = df_plot.reindex(pis, axis=0)
    df_plots.append(df_plot) 
df_scheduling_time_mean_after_first = pd.concat(df_plots, axis=1)
print(df_scheduling_time_mean_after_first.to_latex(na_rep="", float_format="%.0f", escape=False, index_names=False))

## Single table

In [ ]:
df_table_final = pd.concat([df_execution_time, df_scheduling_time_total, df_scheduling_time_mean_after_first])

In [ ]:
print(df_table_final.to_latex(na_rep="", float_format="%.1f", escape=False))

In [ ]:
display(df_table_final.round(4))

## Min and max values

In [ ]:
# execution time
vals = [v for v in df_execution_time.values.reshape(-1) if not np.isnan(v)]
best_vals = sorted(vals)[:3]
worst_vals = sorted(vals, reverse=True)[:3]
print("Best vals:  " + str([f"{v:.1f}" for v in best_vals]))
print("Worst vals: " + str([f"{v:.1f}" for v in worst_vals]))

In [ ]:
# scheduling time (total)
vals = [v for v in df_scheduling_time_total.values.reshape(-1) if not np.isnan(v)]
best_vals = sorted(vals)[:3]
worst_vals = sorted(vals, reverse=True)[:3]
print("Best vals:  " + str([f"{v:.1f}" for v in best_vals]))
print("Worst vals: " + str([f"{v:.1f}" for v in worst_vals]))

In [ ]:
# scheduling time (mean)
vals = [v for v in df_scheduling_time_mean_after_first.values.reshape(-1) if not np.isnan(v)]
best_vals = sorted(vals)[:3]
worst_vals = sorted(vals, reverse=True)[:3]
print("Best vals:  " + str([f"{v:.1f}" for v in best_vals]))
print("Worst vals: " + str([f"{v:.1f}" for v in worst_vals]))

# Scratch